# Demonstration of pytorch


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [2]:
iris = load_iris()

X = iris['data']
y = iris['target']

In [3]:
names = iris['target_names']
feature_names = iris['feature_names']


In [4]:
feature_names

['sepal length (cm)',
 'sepal width (cm)',
 'petal length (cm)',
 'petal width (cm)']

In [5]:
names

array(['setosa', 'versicolor', 'virginica'], dtype='<U10')

In [6]:
df = pd.DataFrame(X,columns=feature_names)
df.head()
# df.tail()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2


In [7]:
df.tail()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
145,6.7,3.0,5.2,2.3
146,6.3,2.5,5.0,1.9
147,6.5,3.0,5.2,2.0
148,6.2,3.4,5.4,2.3
149,5.9,3.0,5.1,1.8


In [8]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(df)

# Split the data set into training and testing
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=2)

In [9]:
print(X_train.shape[0])
print(X_train.shape[1])

120
4


In [10]:
#ANN
import torch                     # use to convert numpy array to pytorch tensor
import torch.nn.functional as F  # use to get activation function
import torch.nn as nn            # use to design neural network
from torch.autograd import Variable  # use to convert numpy array to pytorch tensor

class Model(nn.Module):         # class Model is inherited from base class Module
    def __init__(self, input_dim):
        super(Model, self).__init__()
        self.layer1 = nn.Linear(input_dim, 50)   # Hidden layer1 with 50 neurons 
        self.layer2 = nn.Linear(50, 50)         #  Hidden layer2 with 50 neurons   
        self.layer3 = nn.Linear(50, 3)          # Output layer1 with 3 neurons
        
    def forward(self, x):
        x = F.relu(self.layer1(x))
        x = F.relu(self.layer2(x))
        x = F.softmax(self.layer3(x))               #dim=1
        return x
    
model     = Model(X_train.shape[1])
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
loss_fn   = nn.CrossEntropyLoss()

In [11]:
model

Model(
  (layer1): Linear(in_features=4, out_features=50, bias=True)
  (layer2): Linear(in_features=50, out_features=50, bias=True)
  (layer3): Linear(in_features=50, out_features=3, bias=True)
)

In [12]:
# X_train

In [13]:
import tqdm

EPOCHS  = 100

X_train = Variable(torch.from_numpy(X_train)).float()
y_train = Variable(torch.from_numpy(y_train)).long()
X_test  = Variable(torch.from_numpy(X_test)).float()
y_test  = Variable(torch.from_numpy(y_test)).long()

loss_list     = np.zeros((EPOCHS,))
accuracy_list = np.zeros((EPOCHS,))

for epoch in tqdm.trange(EPOCHS):
    y_pred = model(X_train)
    loss = loss_fn(y_pred, y_train)
    loss_list[epoch] = loss.item()
    
    # Zero gradients
    optimizer.zero_grad()                     
    loss.backward()
    optimizer.step()                          # Update the weight
     
    with torch.no_grad():                     
        y_pred = model(X_test)
        correct = (torch.argmax(y_pred, dim=1) == y_test).type(torch.FloatTensor)
        accuracy_list[epoch] = correct.mean()

100%|██████████| 100/100 [00:00<00:00, 247.37it/s]


In [14]:
print('Accuracy is', accuracy_list[99])

Accuracy is 0.9333333373069763


In [15]:
print('Loss is', loss_list[99])

Loss is 0.6382278800010681


In [16]:
# To save the model
torch.save(model,'iris.pt')

In [17]:
from sklearn.metrics import classification_report

In [18]:
train_pred = model(X_train)
print(train_pred)

tensor([[6.7967e-03, 2.7582e-01, 7.1738e-01],
        [9.7551e-01, 1.6931e-02, 7.5613e-03],
        [3.7555e-02, 9.0916e-01, 5.3287e-02],
        [1.7496e-03, 1.4717e-02, 9.8353e-01],
        [4.7397e-02, 7.3874e-01, 2.1386e-01],
        [9.9564e-01, 2.5889e-03, 1.7704e-03],
        [1.0152e-03, 2.5301e-02, 9.7368e-01],
        [2.0750e-02, 7.3658e-01, 2.4267e-01],
        [2.0338e-03, 9.6649e-01, 3.1475e-02],
        [2.1757e-03, 2.3706e-02, 9.7412e-01],
        [1.7325e-02, 6.9321e-01, 2.8947e-01],
        [1.2558e-02, 8.8334e-01, 1.0411e-01],
        [1.2137e-03, 1.1118e-02, 9.8767e-01],
        [4.6973e-03, 9.8518e-01, 1.0121e-02],
        [9.9309e-01, 4.3306e-03, 2.5744e-03],
        [2.7617e-03, 6.2693e-01, 3.7031e-01],
        [9.8732e-01, 8.6925e-03, 3.9837e-03],
        [1.4932e-02, 8.3054e-01, 1.5453e-01],
        [9.9260e-01, 4.5724e-03, 2.8240e-03],
        [9.9071e-01, 5.8966e-03, 3.3925e-03],
        [9.8533e-01, 1.0038e-02, 4.6299e-03],
        [1.0759e-02, 7.7212e-01, 2

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:17: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


In [19]:
train_pred = model(X_train).argmax(dim=1)

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:17: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


In [20]:
train_pred

tensor([2, 0, 1, 2, 1, 0, 2, 1, 1, 2, 1, 1, 2, 1, 0, 1, 0, 1, 0, 0, 0, 1, 2, 2,
        0, 2, 2, 2, 1, 0, 0, 2, 1, 1, 2, 2, 1, 0, 1, 0, 2, 1, 1, 0, 1, 1, 1, 2,
        0, 1, 0, 1, 2, 0, 1, 0, 0, 0, 2, 2, 0, 0, 2, 2, 1, 2, 1, 1, 2, 0, 2, 2,
        2, 0, 2, 0, 0, 1, 2, 1, 1, 1, 1, 2, 1, 1, 1, 2, 2, 2, 2, 0, 1, 1, 1, 1,
        2, 1, 0, 0, 2, 1, 2, 0, 2, 0, 2, 2, 0, 1, 0, 2, 1, 0, 2, 1, 0, 0, 1, 0])

In [21]:
print(classification_report(y_train,train_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        36
           1       0.91      0.95      0.93        42
           2       0.95      0.90      0.93        42

    accuracy                           0.95       120
   macro avg       0.95      0.95      0.95       120
weighted avg       0.95      0.95      0.95       120



In [22]:
test_pred = model(X_test).argmax(dim=1)

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:17: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


In [23]:
test_pred

tensor([0, 0, 2, 0, 0, 2, 0, 2, 2, 0, 0, 0, 0, 0, 1, 1, 1, 1, 2, 1, 2, 1, 2, 1,
        1, 0, 0, 2, 0, 2])

In [24]:
print(classification_report(y_test,test_pred))

              precision    recall  f1-score   support

           0       1.00      0.93      0.96        14
           1       0.88      0.88      0.88         8
           2       0.89      1.00      0.94         8

    accuracy                           0.93        30
   macro avg       0.92      0.93      0.93        30
weighted avg       0.94      0.93      0.93        30

